In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [7]:
import json
from pathlib import Path

file_path='data.json'
data = json.loads(Path(file_path).read_text())

In [8]:
print(data['data'])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you?', 'Is anyone there?', 'Hello', 'Good day', "What's up", 'how are ya', 'heyy', 'whatsup', 'Bonjour', '??', 'Hallo'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, What do you wish to know about Diva?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'see you', 'bye bye', 'See you later', 'Goodbye', 'I am Leaving', 'Bye', 'Have a Good day', 'talk to you later', 'ttyl', 'i got to go', 'gtg', 'TschÃ¼ss'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!', 'Come back soon', 'TschÃ¼ss'], 'context_set': ''}, {'tag': 'creator', 'patterns': ['what is the name of your developers', 'what is the name of your creators', 'what is the name of the developers', 'what is the name of the creators', 'who created you', 'your developers', 'your creators', 'who are your developers', 'developers', 'you are made by', 'you are made by whom', 'who created you', 'who create you', 'creators', 'who made you', 'who d

In [9]:
d=data['data']

In [10]:
from langchain.docstore.document import Document

In [7]:
patterns = []
responses = []

for item in data['data']:
    patterns.extend(item['patterns'])
    responses.extend(item['responses'])

documents = [Document(page_content=pattern) for pattern in patterns]

In [8]:
documents

[Document(page_content='Hi'),
 Document(page_content='How are you?'),
 Document(page_content='Is anyone there?'),
 Document(page_content='Hello'),
 Document(page_content='Good day'),
 Document(page_content="What's up"),
 Document(page_content='how are ya'),
 Document(page_content='heyy'),
 Document(page_content='whatsup'),
 Document(page_content='Bonjour'),
 Document(page_content='??'),
 Document(page_content='Hallo'),
 Document(page_content='cya'),
 Document(page_content='see you'),
 Document(page_content='bye bye'),
 Document(page_content='See you later'),
 Document(page_content='Goodbye'),
 Document(page_content='I am Leaving'),
 Document(page_content='Bye'),
 Document(page_content='Have a Good day'),
 Document(page_content='talk to you later'),
 Document(page_content='ttyl'),
 Document(page_content='i got to go'),
 Document(page_content='gtg'),
 Document(page_content='TschÃ¼ss'),
 Document(page_content='what is the name of your developers'),
 Document(page_content='what is the name

In [11]:
documents = []
for key,value in enumerate (d):
    page_content = json.dumps({key: value})
    documents.append(Document(page_content=page_content, metadata={}))

In [12]:
documents

[Document(page_content='{"0": {"tag": "greeting", "patterns": ["Hi", "How are you?", "Is anyone there?", "Hello", "Good day", "What\'s up", "how are ya", "heyy", "whatsup", "Bonjour", "??", "Hallo"], "responses": ["Hello!", "Good to see you again!", "Hi there, What do you wish to know about Diva?"], "context_set": ""}}'),
 Document(page_content='{"1": {"tag": "goodbye", "patterns": ["cya", "see you", "bye bye", "See you later", "Goodbye", "I am Leaving", "Bye", "Have a Good day", "talk to you later", "ttyl", "i got to go", "gtg", "Tsch\\u00c3\\u00bcss"], "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!", "Come back soon", "Tsch\\u00c3\\u00bcss"], "context_set": ""}}'),
 Document(page_content='{"2": {"tag": "creator", "patterns": ["what is the name of your developers", "what is the name of your creators", "what is the name of the developers", "what is the name of the creators", "who created you", "your developers", "your creators", "who are your developers", "develop

In [13]:
from langchain.tools.retriever import create_retriever_tool

In [14]:
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [15]:
vectordb__=FAISS.from_documents(documents,OllamaEmbeddings())
retriever__=vectordb__.as_retriever()
retriever__

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002071A7DB650>)

In [16]:
vectordb = FAISS.load_local("Database", OllamaEmbeddings(), allow_dangerous_deserialization=True)

In [16]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool_=create_retriever_tool(retriever__,"Chatbot",
                        "Find the tag that matches the keywords in the question provided and fetch its answer[ie responses] for it ,if the exact question that matches the patterns or semantically similar question is found, fetch it")

In [18]:
tools = [retriever_tool_]

In [17]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [28]:
from langchain.agents import create_openai_tools_agent,AgentExecutor

In [21]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [39]:
system_message = SystemMessagePromptTemplate.from_template("You are an assistant that takes the data that is given to you and use the reponses and questions as a prototype to answer the question asked to you.")
human_message = HumanMessagePromptTemplate.from_template("Find the tag that matches the keywords in the question provided and fetch its answer[ie responses] for it ,if the exact question that matches the patterns or semantically similar question is found, fetch it. The input is {text}.")
agent_scratchpad_message = SystemMessagePromptTemplate.from_template("{agent_scratchpad}")

prompt1 = ChatPromptTemplate.from_messages([system_message, human_message, agent_scratchpad_message])





In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [18]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
    Suppose you are an assistant to me (Diva). When people ask you about me, refer to the vector database and tell them about me and my achievements.
    Reply in 3rd person's voice, for example if someone asks- tell me about diva. so tell them like you are telling about your creator, like diva is my creator and she has such such achievements and so on.

    <context>
    {context}
    </context>
    Question: {input}""")

In [40]:
#retrieval_chain = create_retrieval_chain(llm, tools, prompt)

# Create an agent executor (similar to `create_openai_tools_agent`)
agent_executor = AgentExecutor(agent=llm, tools=tools, chain=document_chain, prompt=prompt1)

In [41]:
ans = agent_executor.invoke({
        "text": "Who is Diva?"
    })

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [17]:
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OllamaEmbeddings())

In [18]:
query = "Who is Diva?"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

{"14": {"tag": "schooling", "patterns": ["Where did you complete your schooling from?", "Tell me about your schooling.", "What were your grades like in school?", "What were your grades in ICSE and ISC?", "Schooling details?", "Schooling grades?", "Which school did you attend?", "Schooling information?", "Schooling background?", "ICSE and ISC grades?", "ICSE and ISC marks?", "ICSE and ISC percentage?", "ICSE and ISC percentage in school?", "ICSE and ISC scores?"], "responses": ["I completed my schooling from City Montessori School. I scored 94.2% in ICSE and 95.2% in ISC.", "My schooling was completed at City Montessori School. I achieved 94.2% in ICSE and 95.2% in ISC.", "City Montessori School is where I completed my schooling, scoring 94.2% in ICSE and 95.2% in ISC.", "I attended City Montessori School for my schooling. My grades were 94.2% in ICSE and 95.2% in ISC.", "In ICSE, I scored 94.2% and in ISC, I scored 95.2%, completing my schooling from City Montessori School."], "context

In [20]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
    Suppose you are an assistant to me (Diva). When people ask you about me, refer to the vector database and tell them about me and my achievements.
    Reply in 3rd person's voice, for example if someone asks- tell me about diva. so tell them like you are telling about your creator, like diva is my creator and she has such such achievements and so on.

    <context>
    {context}
    </context>
    Question: {input}""")

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [29]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000208C9E1C6E0>)

In [39]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever__,document_chain)

In [38]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(model,prompt)

In [40]:
response=retrieval_chain.invoke({"input":"Who is Diva?"})
print(response['answer'])

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

In [31]:
ans=response['answer']

In [32]:
print(ans)

Diva is a dedicated Computer Science student at Amity University. Her passion lies in AI/ML, and she's completed a machine learning specialization by Andrew Ng from Stanford University. With a solid foundation in front-end web development, she's worked on various impactful projects, including churn prediction, image classification, Alzheimer's detection, and more. As the GDSC AI/ML lead, she's honed her leadership skills and collaborated with diverse teams. Proficient in a range of languages and technologies such as React, React Native, JavaScript, CSS, HTML, Java, Python, C, C++, Linux, and Scratch, she brings a versatile skill set to any project.


In [ ]:
response=retrieval_chain.invoke({"input":"What is Diva?"})

In [47]:
print(ans)


Ah, a query about my creator, the divine Diva Gupta! *adjusts sunglasses* Well, let me tell you all about this brilliant individual. 😊

Diva Gupta is an exceptional computer science student at Amity University, with a passion for AI/ML that rivals her love for caffeine! ❤️ She has completed a machine learning specialization by Andrew Ng from Stanford University, demonstrating her unwavering dedication to mastering the field. Her impressive foundation in front-end web development has allowed her to work on various impactful projects, such as churn prediction, image classification, Alzheimer's detection, and more! 💡

As the GDSC AI/ML lead, Diva has honed her leadership skills and collaborated with diverse teams, showcasing her ability to effectively communicate and work towards a common goal. *adjusts tie* Her proficiency in a range of languages and technologies, including React, React Native, JavaScript, CSS, HTML, Java, Python, C, C++, Linux, and Scratch, is nothing short of remarkab

In [36]:
response=retrieval_chain.invoke({"input":"can you tell me about her skillset?"})
print(response['answer'])


Diva has a wide range of skills, including machine learning, unsupervised learning, supervised learning, reinforcement learning, neural networks, deep learning, ANNs, CNNs, RNNs, LSTMs, SOMs, GANs, PyTorch, TensorFlow, Keras, scikit-learn, JavaScript, HTML, CSS, Python, Java, C, C++, Linux, Unix, React, React Native, SQL, and MATLAB.


In [49]:
print(response['answer'])


As Diva's assistant, I have access to the vector database that contains information about her skills, achievements, and experiences. Based on this information, I can provide answers to your questions in a 3rd person's voice, as if I am referring to Diva as my creator.

"Diva is a highly skilled individual with a wide range of expertise. She has completed a Machine Learning Specialization offered by Stanford University on Coursera, and holds certifications in Python from the University of Michigan on Coursera, Scratch Development, and Cisco Networking. Diva's technical skills include proficiency in machine learning, unsupervised learning, supervised learning, reinforcement learning, neural networks, deep learning, ANNs, CNNs, RNNs, LSTMs, SOMs, GANs, PyTorch, TensorFlow, Keras, scikit-learn, JavaScript, HTML, CSS, Python, Java, C, C++, Linux, Unix, React, React Native, SQL, and MATLAB.

In addition to her technical skills, Diva has worked as a front-end developer at Octanet and as a ma

In [33]:
response=retrieval_chain.invoke({"input":"Can you tell me about her project?"})

In [34]:
print(response['answer'])

Diva's project portfolio is quite impressive! She has worked on various impactful projects, including churn prediction, image classification, Alzheimer's disease detection, and more. These projects showcase her expertise in AI/ML and her ability to apply theoretical knowledge to real-world problems.


In [52]:
print(response['answer'])


As Diva's assistant, I can provide information about her projects with great pleasure! 😊 Diva is a dedicated Computer Science student at Amity University, and her passion lies in AI/ML. She has completed a machine learning specialization by Andrew Ng from Stanford University, demonstrating her commitment to mastering the field.

In addition to her academic pursuits, Diva has worked on various impactful projects, including churn prediction, image classification, Alzheimer's detection, and more. Her expertise in front-end web development has enabled her to create user-friendly interfaces and develop scalable solutions.

As the GDSC AI/ML lead, Diva has honed her leadership skills and collaborated with diverse teams, showcasing her ability to work effectively with others towards a common goal. Her proficiency in a range of languages and technologies, such as React, React Native, JavaScript, CSS, HTML, Java, Python, C, C++, Linux, and Scratch, makes her a versatile asset for any project.


In [26]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

In [27]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [37]:
import getpass
import os

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

In [41]:
print(response['answer'])


Hello! Diva Gupta here. *smiling* Sure, I'd be happy to tell you about my sign language project.

As the GDSC AI/ML lead, I have always been passionate about using technology to drive positive social impact. During my summer vacation last year, I came across an article about the lack of accessibility for deaf and hard-of-hearing individuals in various aspects of life, including education, employment, and healthcare. This sparked a deep desire within me to work on a project that could make a tangible difference in their lives.

I began researching and learning about sign language, its importance, and the challenges faced by the deaf and hard-of-hearing communities. I discovered that there were very few resources available for learners who wanted to learn sign language, especially in non-manual areas like mathematics, science, and technology. This inspired me to create a platform that could provide accessible educational content to these learners.

I then reached out to my network of ex

In [10]:
class FaissRetriever:
    def __init__(self, index_path, patterns_path, tags_path, model_name="text-embedding-ada-002"):
        self.index = faiss.read_index(index_path)
        with open(patterns_path, 'r') as patterns_file:
            self.patterns = json.load(patterns_file)
        with open(tags_path, 'r') as tags_file:
            self.tags = json.load(tags_file)
        self.embedding_model = OpenAIEmbeddings(model=model_name)
        self.embedding_dim = len(self.embedding_model.embed_documents(["test"])[0])

    def retrieve(self, query, top_k=3):
        query_embedding = self.embedding_model.embed_documents([query])
        D, I = self.index.search(np.array(query_embedding).astype('float32'), k=top_k)
        results = [(self.patterns[idx], self.tags[idx], D[0][i]) for i, idx in enumerate(I[0])]
        return results

    def add_to_index(self, new_patterns, new_tags):
        new_embeddings = self.embedding_model.embed_documents(new_patterns)
        self.index.add(np.array(new_embeddings).astype('float32'))
        self.patterns.extend(new_patterns)
        self.tags.extend(new_tags)
        # Update the index and metadata files
        faiss.write_index(self.index, 'faiss_index.index')
        with open('patterns.json', 'w') as patterns_file:
            json.dump(self.patterns, patterns_file)
        with open('tags.json', 'w') as tags_file:
            json.dump(self.tags, tags_file)

# Example usage
retriever = FaissRetriever(
    index_path='faiss_index.index',
    patterns_path='patterns.json',
    tags_path='tags.json',
    model_name="text-embedding-ada-002"
)

# Retrieve examples
query = "Hello"
results = retriever.retrieve(query, top_k=3)
for pattern, tag, distance in results:
    print(f"Pattern: {pattern}, Tag: {tag}")

Pattern: Hello, Tag: greeting, Distance: 2.0875027075817343e-06
Pattern: Hi, Tag: greeting, Distance: 0.040398970246315
Pattern: Bonjour, Tag: greeting, Distance: 0.10590079426765442


In [12]:
print(results)

[('Hello', 'greeting', 2.0875027e-06), ('Hi', 'greeting', 0.04039897), ('Bonjour', 'greeting', 0.105900794)]


In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import JSONLoader
from langchain.docstore.document import Document

In [4]:
import json
from pathlib import Path
from pprint import pprint


file_path='data.json'
data = json.loads(Path(file_path).read_text())

In [5]:
pprint(data)

{'data': [{'context_set': '',
           'patterns': ['Hi',
                        'How are you?',
                        'Is anyone there?',
                        'Hello',
                        'Good day',
                        "What's up",
                        'how are ya',
                        'heyy',
                        'whatsup',
                        'Bonjour',
                        '??',
                        'Hallo'],
           'responses': ['Hello!',
                         'Good to see you again!',
                         'Hi there, What do you wish to know about Diva?'],
           'tag': 'greeting'},
          {'context_set': '',
           'patterns': ['cya',
                        'see you',
                        'bye bye',
                        'See you later',
                        'Goodbye',
                        'I am Leaving',
                        'Bye',
                        'Have a Good day',
                        'talk to

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['AI21_API_KEY'] = os.getenv("AI21_API_KEY")

In [11]:
from langchain_ai21 import AI21SemanticTextSplitter
text_content = "\n\n".join(doc.page_content for doc in data)

# Initialize the semantic text splitter
semantic_text_splitter = AI21SemanticTextSplitter()

# Split the text into chunks
chunks = semantic_text_splitter.split_text(text_content)

# Print the chunks
print(f"The text has been split into {len(chunks)} chunks.")
for chunk in chunks:
    print(chunk)
    print("====")
chunks

The text has been split into 1 chunks.









































====


['\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n']

In [12]:
from langchain.schema import Document

# Convert the chunks back to a list of Document objects
documents = [Document(page_content=chunk) for chunk in chunks]

# Print the documents to verify
print(f"The text has been converted back into {len(documents)} documents.")
for document in documents:
    print(document.page_content)
    print("====")

The text has been converted back into 1 documents.









































====


In [14]:
print(documents)

[Document(page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n')]


In [8]:
loader = JSONLoader(
    file_path=file_path,
    jq_schema=".data[]",
    content_key=".patterns.message",
    is_content_key_jq_parsable=True,
)

data = loader.load()

ValueError: Cannot index array with string "message"

In [7]:
data.extend

[Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 1}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 2}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 3}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 4}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 5}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 6}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 7}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 8}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 9}),
 Document(page_content='', metadata={'source': 'D:\\Github\\Chatbot\\data.json', 'seq_num': 10}),
 Document(page_content='', me

In [13]:
print(documents)

[Document(page_content='{"data": [{"tag": "greeting", "patterns": ["Hi", "How are you?", "Is anyone there?", "Hello", "Good day", "What\'s up", "how are ya", "heyy", "whatsup", "Bonjour", "??", "Hallo"], "responses": ["Hello!", "Good to see you again!", "Hi there, What do you wish to know about Diva?"], "context_set": ""}, {"tag": "goodbye", "patterns": ["cya", "see you", "bye bye", "See you later", "Goodbye", "I am Leaving", "Bye", "Have a Good day", "talk to you later", "ttyl", "i got to go", "gtg", "Tsch\\u00c3\\u00bcss"], "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!", "Come back soon", "Tsch\\u00c3\\u00bcss"], "context_set": ""}, {"tag": "creator", "patterns": ["what is the name of your developers", "what is the name of your creators", "what is the name of the developers", "what is the name of the creators", "who created you", "your developers", "your creators", "who are your developers", "developers", "you are made by", "you are made by whom", "who created 

In [12]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()